## Magnitude error from Howell 2006:
## $\sigma_{magnitudes}=\frac{1.0857 \sqrt{N_* + p}}{N_*} $
## $p = n_{pix} (1+ \frac{n_{pix}}{n_B})(N_S + N_D + N_R^2 +G^2\sigma_f^2)$
#### $n_{pix} =$ number of pixels in source region
#### $n_B =$ number of background pixels used to estimate the meal level of the background
#### $N_S =$ total number of photons per pixel from the background/sky
#### $N_D =$ total number of dark current electrons per pixel
#### $N_R^2 =$ total number of electrons per pixel resulting from the read noise
#### $G =$ gain of the CCD
#### $\sigma_f^2 =$ 1 sigma error introduced within the A/D converter ~ 0.289

In [18]:
import numpy as np

# Returns full SNR calculation
def snr(N_star, n_pix, n_B, N_S, N_D, N_R, G, sigma_f=0.289):
    p = n_pix * (1 + n_pix / n_B) * (N_S + N_D + N_R**2 + G**2 * sigma_f**2)
    snr_value = N_star / np.sqrt(N_star + p)
    return snr_value

# Returns SNR estimate based on photon counts
def snr_est(N_star):
    return np.sqrt(N_star)

# Returns magnitude error for a given SNR
def mag_error(snr):
    return 1.0857 / snr

# Returns minimum SNR for a given magnitude error
def min_snr(mag_err):
    return 1.0857 / mag_err

# Returns photon counts per second based on zero point magnitude and target magnitude
def photon_counts_per_sec(zpmag, target_mag):
    return 2.5**(zpmag - target_mag)

## RLMT Estimates

In [20]:
from glob import glob
from astropy.io import fits 
i_images = glob(r"E:\MACRO\MCV\Images\*\*\*_i_*.fts")
i_images_2 = glob(r"E:\MACRO\MCV\Images\*\*_i_*.fts")
print(len(i_images))
print(len(i_images_2))

zpmag = 0
num_included = 0
for img in i_images:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
for img in i_images_2:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1

if num_included > 0:
    print("Average ZPMAG:", zpmag / num_included)
else:
    print("No ZPMAG values found for i fitler in the headers.")

43
3
No ZPMAG values found for i fitler in the headers.


In [19]:
g_images = glob(r"E:\MACRO\MCV\Images\*\*\*_g_*.fts")
g_images_2 = glob(r"E:\MACRO\MCV\Images\*\*_g_*.fts")
print(len(g_images))
print(len(g_images_2))

zpmag = 0
num_included = 0
for img in g_images:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
for img in g_images_2:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
print("Average ZPMAG for g filter:", zpmag / num_included)

6
0
Average ZPMAG for g filter: 21.84211773392965


In [ ]:
r_images = glob(r"E:\MACRO\MCV\Images\*\*\*_r_*.fts")
r_images_2 = glob(r"E:\MACRO\MCV\Images\*\*_r_*.fts")
print(len(r_images))
print(len(r_images_2))

zpmag = 0
num_included = 0
for img in r_images:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
for img in r_images_2:
    header = fits.getheader(img)
    if 'ZPMAG' in header:
        zpmag += header['ZPMAG']
        num_included += 1
print("Average ZPMAG for r filter:", zpmag / num_included)

57
0
Average ZPMAG: 20.839567034245288


## Photon/electron counts for a given mag difference

In [7]:
photon_counts_per_mag_diff_per_sec = [2.5**num for num in range(1, 21)]
photon_counts_per_mag_diff_per_sec = np.array(photon_counts_per_mag_diff_per_sec)
print(photon_counts_per_mag_diff_per_sec)

[2.50000000e+00 6.25000000e+00 1.56250000e+01 3.90625000e+01
 9.76562500e+01 2.44140625e+02 6.10351562e+02 1.52587891e+03
 3.81469727e+03 9.53674316e+03 2.38418579e+04 5.96046448e+04
 1.49011612e+05 3.72529030e+05 9.31322575e+05 2.32830644e+06
 5.82076609e+06 1.45519152e+07 3.63797881e+07 9.09494702e+07]


## Associated mag error estimates

In [8]:
snr_1s = snr_est(photon_counts_per_mag_diff_per_sec * 1)
snr_10s = snr_est(photon_counts_per_mag_diff_per_sec * 10)
snr_30s = snr_est(photon_counts_per_mag_diff_per_sec * 30)
snr_60s = snr_est(photon_counts_per_mag_diff_per_sec * 60)
snr_120s = snr_est(photon_counts_per_mag_diff_per_sec * 120)
snr_300s = snr_est(photon_counts_per_mag_diff_per_sec * 300)

est_mag_errors_1s = mag_error(snr_1s)
est_mag_errors_10s = mag_error(snr_10s) 
est_mag_errors_30s = mag_error(snr_30s)
est_mag_errors_60s = mag_error(snr_60s)
est_mag_errors_120s = mag_error(snr_120s)
est_mag_errors_300s = mag_error(snr_300s)

for i in range(len(snr_1s)):
    print(f"Mag difference: {i+1}, SNR for 1s: {snr_1s[i]:.2f}, Mag error for 1s: {est_mag_errors_1s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 10s: {snr_10s[i]:.2f}, Mag error for 10s: {est_mag_errors_10s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 30s: {snr_30s[i]:.2f}, Mag error for 30s: {est_mag_errors_30s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 60s: {snr_60s[i]:.2f}, Mag error for 60s: {est_mag_errors_60s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 120s: {snr_120s[i]:.2f}, Mag error for 120s: {est_mag_errors_120s[i]:.4f}")
    print(f"Mag difference: {i+1}, SNR for 300s: {snr_300s[i]:.2f}, Mag error for 300s: {est_mag_errors_300s[i]:.4f}")
    print("")

Mag difference: 1, SNR for 1s: 1.58, Mag error for 1s: 0.6867
Mag difference: 1, SNR for 10s: 5.00, Mag error for 10s: 0.2171
Mag difference: 1, SNR for 30s: 8.66, Mag error for 30s: 0.1254
Mag difference: 1, SNR for 60s: 12.25, Mag error for 60s: 0.0886
Mag difference: 1, SNR for 120s: 17.32, Mag error for 120s: 0.0627
Mag difference: 1, SNR for 300s: 27.39, Mag error for 300s: 0.0396

Mag difference: 2, SNR for 1s: 2.50, Mag error for 1s: 0.4343
Mag difference: 2, SNR for 10s: 7.91, Mag error for 10s: 0.1373
Mag difference: 2, SNR for 30s: 13.69, Mag error for 30s: 0.0793
Mag difference: 2, SNR for 60s: 19.36, Mag error for 60s: 0.0561
Mag difference: 2, SNR for 120s: 27.39, Mag error for 120s: 0.0396
Mag difference: 2, SNR for 300s: 43.30, Mag error for 300s: 0.0251

Mag difference: 3, SNR for 1s: 3.95, Mag error for 1s: 0.2747
Mag difference: 3, SNR for 10s: 12.50, Mag error for 10s: 0.0869
Mag difference: 3, SNR for 30s: 21.65, Mag error for 30s: 0.0501
Mag difference: 3, SNR for 

## $\Delta m = 21 - 2.5log_{10}(photons/s \pm transit\ depth \%)$

In [16]:
transit_depths = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 50.0]  # in percentage
delta_mag_results = {}
for depth in transit_depths:
    print(f"Transit depth: {depth}%")
    for i, counts in enumerate(photon_counts_per_mag_diff_per_sec):
        m = 21 - 2.5 * np.log10(counts)
        # delta_m_P = 21 - 2.5 * np.log10(counts * (1 + depth / 100)) - m
        delta_m_M = 21 - 2.5 * np.log10(counts * (1 - depth / 100)) - m

        # print(f"Mag difference: {i+1}, Δm+ : {delta_m_P:.4f}, Δm- : {delta_m_M:.4f}, m: {m:.4f}")
        print(f"Δm- : {delta_m_M:.4f}")
        if i == 0:
            delta_mag_results[depth] = {"delta_m_M": [delta_m_M], "m": [m], "mag_diff": [i+1], "transit_depths": [depth], "photon_counts": [counts]}
            break
    print("")

Transit depth: 0.01%
Δm- : 0.0001

Transit depth: 0.1%
Δm- : 0.0011

Transit depth: 0.5%
Δm- : 0.0054

Transit depth: 1.0%
Δm- : 0.0109

Transit depth: 2.0%
Δm- : 0.0219

Transit depth: 5.0%
Δm- : 0.0557

Transit depth: 10.0%
Δm- : 0.1144

Transit depth: 25.0%
Δm- : 0.3123

Transit depth: 50.0%
Δm- : 0.7526



## Estimating 10% flux on center pixel

In [10]:
full_well = 65535.0
counts_per_sec_10_percent = photon_counts_per_mag_diff_per_sec*0.1
saturation_times = full_well / counts_per_sec_10_percent
for i, time in enumerate(saturation_times):
    print(f"Mag difference: {i+1}, Saturation time (s): {time:.2f}")

Mag difference: 1, Saturation time (s): 262140.00
Mag difference: 2, Saturation time (s): 104856.00
Mag difference: 3, Saturation time (s): 41942.40
Mag difference: 4, Saturation time (s): 16776.96
Mag difference: 5, Saturation time (s): 6710.78
Mag difference: 6, Saturation time (s): 2684.31
Mag difference: 7, Saturation time (s): 1073.73
Mag difference: 8, Saturation time (s): 429.49
Mag difference: 9, Saturation time (s): 171.80
Mag difference: 10, Saturation time (s): 68.72
Mag difference: 11, Saturation time (s): 27.49
Mag difference: 12, Saturation time (s): 10.99
Mag difference: 13, Saturation time (s): 4.40
Mag difference: 14, Saturation time (s): 1.76
Mag difference: 15, Saturation time (s): 0.70
Mag difference: 16, Saturation time (s): 0.28
Mag difference: 17, Saturation time (s): 0.11
Mag difference: 18, Saturation time (s): 0.05
Mag difference: 19, Saturation time (s): 0.02
Mag difference: 20, Saturation time (s): 0.01


## Estimating 25% flux on center pixel

In [11]:
saturation_times_25 = full_well / (photon_counts_per_mag_diff_per_sec*0.25)
for i, time in enumerate(saturation_times_25):
    print(f"Mag difference: {i+1}, Saturation time (s) at 25% flux: {time:.2f}")

Mag difference: 1, Saturation time (s) at 25% flux: 104856.00
Mag difference: 2, Saturation time (s) at 25% flux: 41942.40
Mag difference: 3, Saturation time (s) at 25% flux: 16776.96
Mag difference: 4, Saturation time (s) at 25% flux: 6710.78
Mag difference: 5, Saturation time (s) at 25% flux: 2684.31
Mag difference: 6, Saturation time (s) at 25% flux: 1073.73
Mag difference: 7, Saturation time (s) at 25% flux: 429.49
Mag difference: 8, Saturation time (s) at 25% flux: 171.80
Mag difference: 9, Saturation time (s) at 25% flux: 68.72
Mag difference: 10, Saturation time (s) at 25% flux: 27.49
Mag difference: 11, Saturation time (s) at 25% flux: 10.99
Mag difference: 12, Saturation time (s) at 25% flux: 4.40
Mag difference: 13, Saturation time (s) at 25% flux: 1.76
Mag difference: 14, Saturation time (s) at 25% flux: 0.70
Mag difference: 15, Saturation time (s) at 25% flux: 0.28
Mag difference: 16, Saturation time (s) at 25% flux: 0.11
Mag difference: 17, Saturation time (s) at 25% flux:

## Peak pixel at ~1.03-1.07 airmass appears to range around 1.5-4% actually, so our times can be longer

In [51]:
import pandas as pd
data = pd.read_csv(r"E:\school\2025 Fall\Observational Astro\SNR - Mag goals\AN_UMa_r-i_few_stars.csv")

t1_peaks = np.array(data['Peak_T1'].values)
t1_sky = np.array(data['Source-Sky_T1'].values)

c2_peaks = np.array(data['Peak_C2'].values)
c2_sky = np.array(data['Source-Sky_C2'].values)

c3_peaks = np.array(data['Peak_C3'].values)
c3_sky = np.array(data['Source-Sky_C3'].values)

c4_peaks = np.array(data['Peak_C4'].values)
c4_sky = np.array(data['Source-Sky_C4'].values)

c5_peaks = np.array(data['Peak_C5'].values)
c5_sky = np.array(data['Source-Sky_C5'].values)

t1_peak_percent = (t1_peaks / t1_sky * 100)
c2_peak_percent = (c2_peaks / c2_sky * 100)
c3_peak_percent = (c3_peaks / c3_sky * 100)
c4_peak_percent = (c4_peaks / c4_sky * 100)
c5_peak_percent = (c5_peaks / c5_sky * 100)

mean_t1 = np.mean(t1_peak_percent)
mean_c2 = np.mean(c2_peak_percent)
mean_c3 = np.mean(c3_peak_percent)
mean_c4 = np.mean(c4_peak_percent)
mean_c5 = np.mean(c5_peak_percent)

max_t1 = np.max(t1_peak_percent)
max_c2 = np.max(c2_peak_percent)
max_c3 = np.max(c3_peak_percent)
max_c4 = np.max(c4_peak_percent)
max_c5 = np.max(c5_peak_percent)

min_t1 = np.min(t1_peak_percent)
min_c2 = np.min(c2_peak_percent)    
min_c3 = np.min(c3_peak_percent)
min_c4 = np.min(c4_peak_percent)
min_c5 = np.min(c5_peak_percent)

print("T1 Mean Peak %:", mean_t1, "Max Peak %:", max_t1, "Min Peak %:", min_t1)
print("C2 Mean Peak %:", mean_c2, "Max Peak %:", max_c2, "Min Peak %:", min_c2)
print("C3 Mean Peak %:", mean_c3, "Max Peak %:", max_c3, "Min Peak %:", min_c3)
print("C4 Mean Peak %:", mean_c4, "Max Peak %:", max_c4, "Min Peak %:", min_c4)
print("C5 Mean Peak %:", mean_c5, "Max Peak %:", max_c5, "Min Peak %:", min_c5)

T1 Mean Peak %: 2.758971714267329 Max Peak %: 4.340521685092317 Min Peak %: 2.0551799152155112
C2 Mean Peak %: 3.5604927006260594 Max Peak %: 5.56713092796591 Min Peak %: 2.1630258074712794
C3 Mean Peak %: 3.352638136970748 Max Peak %: 5.226239823192906 Min Peak %: 2.0284122547773014
C4 Mean Peak %: 3.507246821022393 Max Peak %: 5.357621243661197 Min Peak %: 2.1789910265671772
C5 Mean Peak %: 3.607076624617365 Max Peak %: 5.520219832131214 Min Peak %: 2.2634184378564015


## Better estimates

In [54]:
full_well = 65535.0
counts_per_sec_2_percent = photon_counts_per_mag_diff_per_sec*0.02
saturation_times_2_percent = full_well / counts_per_sec_2_percent
for i, time in enumerate(saturation_times_2_percent):
    print(f"Mag difference at 2%: {i+1}, Saturation time (s): {time:.2f}")

print("")

counts_per_sec_6_percent = photon_counts_per_mag_diff_per_sec*0.06
saturation_times_6_percent = full_well / counts_per_sec_6_percent
for i, time in enumerate(saturation_times_6_percent):
    print(f"Mag difference at 6%: {i+1}, Saturation time (s): {time:.2f}")

Mag difference at 2%: 1, Saturation time (s): 1310700.00
Mag difference at 2%: 2, Saturation time (s): 524280.00
Mag difference at 2%: 3, Saturation time (s): 209712.00
Mag difference at 2%: 4, Saturation time (s): 83884.80
Mag difference at 2%: 5, Saturation time (s): 33553.92
Mag difference at 2%: 6, Saturation time (s): 13421.57
Mag difference at 2%: 7, Saturation time (s): 5368.63
Mag difference at 2%: 8, Saturation time (s): 2147.45
Mag difference at 2%: 9, Saturation time (s): 858.98
Mag difference at 2%: 10, Saturation time (s): 343.59
Mag difference at 2%: 11, Saturation time (s): 137.44
Mag difference at 2%: 12, Saturation time (s): 54.97
Mag difference at 2%: 13, Saturation time (s): 21.99
Mag difference at 2%: 14, Saturation time (s): 8.80
Mag difference at 2%: 15, Saturation time (s): 3.52
Mag difference at 2%: 16, Saturation time (s): 1.41
Mag difference at 2%: 17, Saturation time (s): 0.56
Mag difference at 2%: 18, Saturation time (s): 0.23
Mag difference at 2%: 19, Satur

## SNR where $\Delta m = \sigma_m$

In [60]:
min_snrs = []
snr_needed = []

for i in delta_mag_results:
    snr_val = min_snr(delta_mag_results[i]["delta_m_M"][0])
    snr_goal = snr_val * 3

    snr_val = round(snr_val, 2)
    snr_goal = round(snr_goal, 2)

    min_snrs.append(snr_val)
    snr_needed.append(snr_goal)

acceptable_snrs = [[], [], [], [], [], [], [], [], []]  # (SNR, exposure time, mag difference from zero point)
exposures = [1, 10, 30, 60, 120, 300]
for i, snr in enumerate(snr_needed):
    for t, snrs in enumerate([snr_1s, snr_10s, snr_30s, snr_60s, snr_120s, snr_300s]):
        for j, actual_snr in enumerate(snrs):
            if snr <= actual_snr and exposures[t] <= saturation_times_6_percent[j]:
                acceptable_snrs[i].append((actual_snr, exposures[t], j + 1))

for i in range(len(acceptable_snrs)):
    print(f"Number of acceptable SNRs for transit depth {transit_depths[i]}%: {len(acceptable_snrs[i])}")

print("")

for i, depth in enumerate(transit_depths):
    if len(acceptable_snrs[i]) == 0:
        print(f"No acceptable exposure times found for transit depth {depth}%")
        print("")
    else:    
        print(f"Transit depth: {depth}%, SNR at Δm = σ_m: {min_snrs[i]}, SNR needed for 3σ detection: {snr_needed[i]}")
        print("Acceptable combinations:")
        for snr in acceptable_snrs[i]:
            print(f"SNR: {snr[0]:.2f}, Exposure time: {snr[1]}s, Mag difference from zero point: {snr[2]}, Mag (~g zpmag = 22): {22 - snr[2]}, Mag (~i zpmag = 19): {19 - snr[2]}")
        print("")



Number of acceptable SNRs for transit depth 0.01%: 0
Number of acceptable SNRs for transit depth 0.1%: 0
Number of acceptable SNRs for transit depth 0.5%: 7
Number of acceptable SNRs for transit depth 1.0%: 16
Number of acceptable SNRs for transit depth 2.0%: 25
Number of acceptable SNRs for transit depth 5.0%: 37
Number of acceptable SNRs for transit depth 10.0%: 46
Number of acceptable SNRs for transit depth 25.0%: 57
Number of acceptable SNRs for transit depth 50.0%: 62

No acceptable exposure times found for transit depth 0.01%

No acceptable exposure times found for transit depth 0.1%

Transit depth: 0.5%, SNR at Δm = σ_m: 199.49, SNR needed for 3σ detection: 598.48
Acceptable combinations:
SNR: 610.35, Exposure time: 1s, Mag difference from zero point: 14, Mag (~g zpmag = 22): 8, Mag (~i zpmag = 19): 5
SNR: 965.05, Exposure time: 1s, Mag difference from zero point: 15, Mag (~g zpmag = 22): 7, Mag (~i zpmag = 19): 4
SNR: 772.04, Exposure time: 10s, Mag difference from zero point: 